In [ ]:
! pip install transformers datasets torch sklearn


In [43]:
import torch
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score


# Load IMDb Dataset

In [44]:
dataset = load_dataset("imdb")


# Tokenization:

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id  

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

train_dataset = dataset['train'].map(tokenize_function, batched=True)
test_dataset = dataset['test'].map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


# Evaluation Metrics (Accuracy):

In [51]:
import torch
from sklearn.metrics import accuracy_score

def compute_metrics(p):
    preds, labels = p
    preds = torch.tensor(preds) if not isinstance(preds, torch.Tensor) else preds
    preds = torch.argmax(preds, axis=1)  
    return {"accuracy": accuracy_score(labels, preds)}


# Set Up Training Arguments:

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',           
    num_train_epochs=1,                
    per_device_train_batch_size=8,     
    per_device_eval_batch_size=8,      
    warmup_steps=500,                 
    weight_decay=0.01,                 
    logging_dir='./logs',              
    logging_steps=10,                  
    evaluation_strategy="epoch",       
    save_strategy="epoch",             
    load_best_model_at_end=True,       
)

# Initialize the Trainer:

In [53]:
train_subset = train_dataset.select(range(200))
test_subset = test_dataset.select(range(200))

In [54]:
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_subset,         
    eval_dataset=test_subset,           
    compute_metrics=compute_metrics      
)


# Train the Model:

In [ ]:
trainer.train()


# Evaluate the Model:

In [ ]:
eval_results = trainer.evaluate()

print(f"Test Loss: {eval_results['eval_loss']:.4f}")
print(f"Test Accuracy: {eval_results['eval_accuracy']:.2f}")


# Make Predictions:

In [ ]:
def predict(texts):
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**encodings)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions

texts = ["This movie was amazing!", "I hated this movie."]
predictions = predict(texts)

pred_labels = ["positive" if label == 1 else "negative" for label in predictions]
print(pred_labels)
